In [165]:
import pandas as pd
obj=pd.read_csv('third_dataset_09_12_2018.csv')
len(obj)

83989

In [159]:
#first check whether there is invalide data(like wrong elements symbol) or 
#(repeat elements in the formula) impossible case there....

#At the same time, count the elemnts frequencies(show up once in a compound, add once in the dictionary)

from pymatgen import Composition, Element

invalide_data_N=0;
invalide_data_material_id=[]
stat_elements={}

#id_clean1 store the material_id for the types of elements are 2,3,or 4;
id_clean1=[]
for i in range(len(obj)):
    #print(i," ")
    data_row=obj[i:i+1]
    #print(data_row)
    if(type(data_row["pretty_formula"][i])==float):
        invalide_data_material_id.append(data_row["material_id"])
        continue
    comp=Composition(data_row["pretty_formula"][i])
    for j in comp:
        #testing whether j is in periodic table, it will produce error if not
        test_elemnt=Element(j)
        #count the frequency of element
        if j in stat_elements:
            stat_elements[j]+=1
        else:
            stat_elements[j]=1
    if len(comp)>4 or len(comp)<2:
        continue
    else:
        id_clean1.append(data_row["material_id"][i])


In [160]:
print(len(id_clean1))
print(len(stat_elements))

75841
89


In [161]:
id_clean1[:10]

['mp-1000',
 'mp-10000',
 'mp-10003',
 'mp-10004',
 'mp-1000440',
 'mp-1000487',
 'mp-1000588',
 'mp-10006',
 'mp-1000625',
 'mp-1000712']

In [158]:
obj[0:1]["material_id"]

0    mp-1
Name: material_id, dtype: object

In [162]:
from pymatgen import MPRester                                                     
import urllib.request                                                             
import json

                                                                                  
if __name__ == "__main__":                                                        
    MAPI_KEY = "zQIrujyVEwEhTBRW"  # You must change this to your Materials API key! (or set MAPI_KEY env variable)                                                              
                                                                                  
    with MPRester(MAPI_KEY) as m: # object for connecting to MP Rest interface 
        criteria={'material_id': {'$in':id_clean1[:]}} # to avoid straining the servers, this is only using the first 4 materials
        #properties=['energy', 'pretty_formula','total_electrons','density']
        properties=['material_id','pretty_formula'] 
        # list a few quanteties of interest
        data = m.query(criteria, properties)                                      
        print(data[:10])

[{'material_id': 'mp-1000', 'pretty_formula': 'BaTe'}, {'material_id': 'mp-10000', 'pretty_formula': 'Hf2S'}, {'material_id': 'mp-10003', 'pretty_formula': 'Nb4CoSi'}, {'material_id': 'mp-10004', 'pretty_formula': 'Mo3P'}, {'material_id': 'mp-1000440', 'pretty_formula': 'CrN'}, {'material_id': 'mp-1000487', 'pretty_formula': 'CeAlO3'}, {'material_id': 'mp-1000588', 'pretty_formula': 'U2IrC2'}, {'material_id': 'mp-10006', 'pretty_formula': 'TlAgTe2'}, {'material_id': 'mp-1000625', 'pretty_formula': 'K(CoSe)2'}, {'material_id': 'mp-1000712', 'pretty_formula': 'RbYbS2'}]


In [164]:
import csv
keys=data[0].keys()
with open('data_clean1_09_12_2018.csv','w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

In [168]:
obj_clean1=pd.read_csv('data_clean1_09_12_2018.csv')
print(len(obj_clean1))

75841


In [171]:
obj_clean1.head()

,material_id,pretty_formula
0,mp-1000,BaTe
1,mp-10000,Hf2S
2,mp-10003,Nb4CoSi
3,mp-10004,Mo3P
4,mp-1000440,CrN


In [196]:
print(len(stat_elements))
import operator
sorted_elements = sorted(stat_elements.items(), key=operator.itemgetter(1),reverse=True)

#you may uncomment to see the frequencies of elements
#print(sorted_elements)

89
[(Element O, 43334), (Element Li, 13943), (Element P, 9872), (Element Mg, 9818), (Element Mn, 7157), (Element Fe, 6963), (Element F, 6960), (Element S, 6336), (Element Si, 6083), (Element V, 5484), (Element H, 5289), (Element Cu, 5216), (Element Co, 5025), (Element Na, 4824), (Element C, 4822), (Element N, 4796), (Element Ni, 4716), (Element Ca, 4449), (Element Ti, 4132), (Element Ba, 4094), (Element Cr, 4057), (Element K, 3954), (Element B, 3765), (Element Al, 3742), (Element Zn, 3724), (Element Se, 3486), (Element Sn, 3435), (Element Sr, 3199), (Element Sb, 3130), (Element Cl, 3096), (Element W, 2859), (Element Mo, 2828), (Element Te, 2823), (Element Bi, 2756), (Element Ge, 2712), (Element La, 2616), (Element Y, 2507), (Element Nb, 2323), (Element Rb, 2311), (Element Ga, 2173), (Element As, 2098), (Element Ag, 2093), (Element Cs, 2047), (Element In, 1999), (Element Cd, 1970), (Element Ce, 1739), (Element Pd, 1712), (Element Zr, 1654), (Element Tl, 1644), (Element I, 1594), (Elemen

In [197]:
#test the form of the sorted elements

#print(sorted_elements)
a=str(sorted_elements[0][0])
if sorted_elements[0][0]==Element("Ne"):
    print("yay")
#len(sorted_elements)

89

In [201]:
#if we pick first 80 elements, the remaining will be wiped. [0:80] will be saved, [80:] will be wiped
#K can be changed for adjusting parameter
K=80
wiping=[]
for i in range(80,len(sorted_elements)):
    wiping.append(sorted_elements[i][0])

#you can check the wiped elements 
print(wiping)

[Element Pm, Element Np, Element Pa, Element Ac, Element Xe, Element Kr, Element He, Element Ar, Element Ne]


In [202]:
#Second cleaning, just get the first 80 elements

from pymatgen import Composition, Element

#id_clean2 store the material_id for the most K(80 in default value) elements;
id_clean2=[]
for i in range(len(obj_clean1)):
    data_row=obj_clean1[i:i+1]
    #print(data_row)
    comp=Composition(data_row["pretty_formula"][i])
    F=0
    for j in comp:
        #testing whether j is in our wiping list, if break present cycle and set value F=1
        if Element(j) in wiping:
            F=1
            break
        #count the frequency of element
    if F:
        print("out")
        continue
    else:
        id_clean2.append(data_row["material_id"][i])


out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out
out


In [203]:
len(id_clean2)

75056

In [204]:
from pymatgen import MPRester                                                     
import urllib.request                                                             
import json

                                                                                  
if __name__ == "__main__":                                                        
    MAPI_KEY = "zQIrujyVEwEhTBRW"  # You must change this to your Materials API key! (or set MAPI_KEY env variable)                                                              
                                                                                  
    with MPRester(MAPI_KEY) as m: # object for connecting to MP Rest interface 
        criteria={'material_id': {'$in':id_clean2[:]}} # to avoid straining the servers, this is only using the first 4 materials
        #properties=['energy', 'pretty_formula','total_electrons','density']
        properties=['material_id','pretty_formula'] 
        # list a few quanteties of interest
        data = m.query(criteria, properties)                                      
        print(data[:10])

[{'material_id': 'mp-1000', 'pretty_formula': 'BaTe'}, {'material_id': 'mp-10000', 'pretty_formula': 'Hf2S'}, {'material_id': 'mp-10003', 'pretty_formula': 'Nb4CoSi'}, {'material_id': 'mp-10004', 'pretty_formula': 'Mo3P'}, {'material_id': 'mp-1000440', 'pretty_formula': 'CrN'}, {'material_id': 'mp-1000487', 'pretty_formula': 'CeAlO3'}, {'material_id': 'mp-1000588', 'pretty_formula': 'U2IrC2'}, {'material_id': 'mp-10006', 'pretty_formula': 'TlAgTe2'}, {'material_id': 'mp-1000625', 'pretty_formula': 'K(CoSe)2'}, {'material_id': 'mp-1000712', 'pretty_formula': 'RbYbS2'}]


In [205]:
import csv
keys=data[0].keys()
with open('data_clean2_09_12_2018.csv','w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)